In [ ]:
# !pip install pandas
# !pip install matplotlib
# !pip install scipy
# !pip install vincent
# !pip install zipcode
# !pip install tqdm
# !pip install gmplot
# !pip install sklearn

In [2]:
import pandas
df = pandas.read_csv("loan.csv")

/home/louis/projects/da_project/.env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
df.groupby("loan_status").size()

loan_status
Charged Off                                             45248
Current                                                601779
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Does not meet the credit policy. Status:Fully Paid       1988
Fully Paid                                             207723
In Grace Period                                          6253
Issued                                                   8460
Late (16-30 days)                                        2357
Late (31-120 days)                                      11591
dtype: int64

In [4]:
df = df.iloc[1:100000]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 1 to 99999
Data columns (total 74 columns):
id                             99999 non-null int64
member_id                      99999 non-null int64
loan_amnt                      99999 non-null float64
funded_amnt                    99999 non-null float64
funded_amnt_inv                99999 non-null float64
term                           99999 non-null object
int_rate                       99999 non-null float64
installment                    99999 non-null float64
grade                          99999 non-null object
sub_grade                      99999 non-null object
emp_title                      93648 non-null object
emp_length                     96253 non-null object
home_ownership                 99999 non-null object
annual_inc                     99995 non-null float64
verification_status            99999 non-null object
issue_d                        99999 non-null object
loan_status                    99999 no

In [6]:
to_drop = ["id", "member_id", "url"]
to_drop.extend(["addr_state","zip_code"])
to_drop.append("desc")
to_drop.extend(['funded_amnt', 'funded_amnt_inv'])
to_drop.extend(['total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp'])
to_drop.extend(['mths_since_last_delinq', 'mths_since_last_record'])
to_drop.append('emp_title')
to_drop.extend(['last_pymnt_d', 'last_pymnt_amnt'])

In [7]:
# df["loan_status"].unique()
def default_or_not(x):
    if x == "Charged Off":
        return 1
    if x == "Fully Paid":
        return 0
    else:
        return None
    
df["default"] = df["loan_status"].apply(default_or_not)
df = df[df["default"].notnull()]
to_drop.append("loan_status")

In [8]:
# df["term"].unique()
df["long_term"] = (df["term"] == ' 60 months').apply(int)
to_drop.append("term")

In [9]:
def emp_year(x):
    try:
        x = x.split()[0]
    finally:
        if x == '<':
            return 0
        if x == '10+':
            return 10
        else:
            return int(x)
        
df = df[df["emp_length"].notnull()]
df["emp_year"] = df["emp_length"].apply(emp_year)
to_drop.append("emp_length")

In [10]:
# single value variable
for k in df.keys():
    #print(k, len(df[k].unique()))
    if len(df[k].unique()) == 1:
        to_drop.append(k)

In [11]:
print(df['grade'].unique())
df['grade_a'] = (df['grade'] == 'A').apply(int)
df['grade_b'] = (df['grade'] == 'B').apply(int)
df['grade_c'] = (df['grade'] == 'C').apply(int)
df['grade_d'] = (df['grade'] == 'D').apply(int)
df['grade_e'] = (df['grade'] == 'E').apply(int)
df['grade_f'] = (df['grade'] == 'F').apply(int)
df['grade_g'] = (df['grade'] == 'G').apply(int)
to_drop.append("grade")
to_drop.append("sub_grade")

['C' 'A' 'E' 'F' 'B' 'D' 'G']


In [12]:
df.groupby('home_ownership').count()
df['own_home'] = (df['home_ownership'] == "OWN").apply(int)
to_drop.append('home_ownership')

In [13]:
df['pymnt_plan'].unique()

array(['n', 'y'], dtype=object)

In [14]:
print(to_drop)

['id', 'member_id', 'url', 'addr_state', 'zip_code', 'desc', 'funded_amnt', 'funded_amnt_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'mths_since_last_delinq', 'mths_since_last_record', 'emp_title', 'last_pymnt_d', 'last_pymnt_amnt', 'loan_status', 'term', 'emp_length', 'out_prncp', 'out_prncp_inv', 'next_pymnt_d', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'grade', 'sub_grade', 'home_ownership']


In [15]:
print(to_drop)
df = df.drop(columns = to_drop, errors = 'ignore')

['id', 'member_id', 'url', 'addr_state', 'zip_code', 'desc', 'funded_amnt', 'funded_amnt_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'mths_since_last_delinq', 'mths_since_last_record', 'emp_title', 'last_pymnt_d', 'last_pymnt_amnt', 'loan_status', 'term', 'emp_length', 'out_prncp', 'out_prncp_inv', 'next_pymnt_d', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'grade', 'sub_grade', 'home_ownership']


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62288 entries, 1 to 99998
Data columns (total 39 columns):
loan_amnt                      62288 non-null float64
int_rate                       62288 non-null float64
installment                    62288 non-null float64
annual_inc                     62288 non-null float64
verification_status            62288 non-null object
issue_d                        62288 non-null object
pymnt_plan                     62288 non-null object
purpose                        62288 non-null object
title                          62278 non-null object
dti                            62288 non-null float64
delinq_2yrs                    62288 non-null float64
earliest_cr_line               62288 non-null object
inq_last_6mths                 62288 non-null float64
open_acc                       62288 non-null float64
pub_rec                        62288 non-null float64
revol_bal                      62288 non-null float64
revol_util                     62

In [189]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

X = df[['long_term','emp_year', 'own_home', 'int_rate', 'grade_a', 'grade_b', 'grade_c', 'grade_d', 'grade_e', 'grade_f', 'dti']]
y = df['default']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
weight = y_train*4 + 1 

logreg = LogisticRegression()
# logreg.fit(X_train, y_train)
logreg.fit(X_train, y_train, sample_weight=weight)

y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.63


In [190]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[9756 5850]
 [1093 1988]]


In [191]:
y_p = logreg.predict_proba(X_test)[:,0] > X_test['int_rate']/(100+X_test['int_rate'])

In [192]:
y_pr = logreg.predict_proba(X_test)[:,0] > 0.5

In [193]:
sum(y_pr)

10849

In [194]:
matrix = confusion_matrix(y_test, y_p)
print(matrix)

[[  166 15440]
 [  115  2966]]


In [188]:
logreg.predict_proba(X_test)[1:10,0]

array([ 0.77536626,  0.89539697,  0.48409131,  0.89201621,  0.69050531,
        0.84481952,  0.81266717,  0.45372922,  0.50646895])

In [ ]:
df.head(5)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
df["loan_amnt"].plot.density()

In [ ]:
df.groupby("grade").size().plot.bar()

In [ ]:
df.groupby("grade")["int_rate"].mean()

In [ ]:
df.groupby("sub_grade")["int_rate"].mean().plot.bar()

In [ ]:
df.groupby("emp_length")["id"].count().plot.bar()

In [ ]:
def get_loc(zip_code):
    import zipcode
    short_code = zip_code.replace('x', '')
    l = zipcode.islike(short_code)
    if l:
        location = l[0]
        return (location.lon, location.lat)
    else:
        return (None, None)

from tqdm import tqdm
tqdm.pandas(desc="Pandas progress")
df["lon"] = df["zip_code"].progress_apply(lambda x: get_loc(x)[0])
df["lat"] = df["zip_code"].progress_apply(lambda x: get_loc(x)[1])

import gmplot
df2 = df[df["lon"].notnull()]
gmap = gmplot.GoogleMapPlotter.from_geocode("The United State",4)
gmap.heatmap(df2['lat'], df2['lon'])
gmap.draw('heat_map.html')

In [ ]:
df.groupby("loan_status").size().plot.bar()

In [ ]:
df["loan_status"].unique()

https://www.dataquest.io/blog/machine-learning-preparing-data/

In [ ]:
df["annual_inc"].plot.density(logx = True)

In [ ]:
df["annual_inc"].describe()

In [ ]:
df.iloc[3]

In [ ]:
df.groupby("purpose").size().plot.bar()

In [ ]:
df['dti'].plot.density()

In [ ]:
df["verification_status"].unique()

In [ ]:
df[df["annual_inc"] > 1000000]

In [ ]:
df.groupby("loan_status")["int_rate"].mean().plot.bar()

In [ ]:
df.groupby(("term","loan_status")).size().plot.bar()

In [ ]:
df["emp_title"].unique()